# Import

In [1]:
import sys
import os
from datetime import datetime

root_dir = '../../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
import pandas as pd

pd.set_option('display.max_columns', 100)

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

# Output settings

In [2]:
out_to_file = True
out_dir = '../../out/comparison/ml_100k/'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Dataset

In [3]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
n_feedback = dataset.n_pos_pairs
train_set, test_set = dataset.get_train_and_test_set(neg_pair_weight=10)

In [4]:
print(f'n_user = {n_user}')
print(f'n_item = {n_item}')
print(f'n_feedback = {n_feedback}')

n_user = 940
n_item = 1447
n_feedback = 55369


# Device

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

In [6]:
print(device)

cpu


# Evaluator

In [7]:
# The size of recommendation set (K)
ks = [10]

score_function_dict = {
    "Recall"       : evaluators.recall,
    "Unpopularity" : evaluators.unpopularity,
    "Serendipity"  : evaluators.serendipity,
    "Long-tail rate": evaluators.longtail_rate,
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)

# Sampler

In [8]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# Model

In [9]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
no_progressbar = False
valid_per_epoch = 10
n_sample = 30
bias = 0.8
gamma = distributions.Gamma()


model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [10]:
mp = searches.MutualProximity(model, gamma, n_sample, bias)

In [11]:
trainer.fit(n_batch, n_epoch, mp, userwise, valid_per_epoch)

100%|██████████| 940/940 [00:15<00:00, 61.03it/s]


In [12]:
trainer.valid_scores

,Recall@10,Unpopularity@10,Serendipity@10,Long-tail rate@10,epoch,losses
0,0.124080,311.274128,3.130736,0.769255,0,NaN
0,0.268720,104.950392,3.971679,0.369681,10,0.473153
0,0.443104,44.724745,5.352079,0.278511,20,0.303267
0,0.497399,41.721191,7.216402,0.288511,30,0.259052
0,0.519735,43.737460,8.129318,0.302447,40,0.240115
0,0.529289,45.277515,8.637594,0.310745,50,0.236321


In [13]:
mp2 = searches.MutualProximity2(model, gamma, n_sample, bias)

In [14]:
trainer.fit(n_batch, n_epoch, mp2, userwise, valid_per_epoch)

100%|██████████| 940/940 [00:15<00:00, 61.04it/s]


In [15]:
trainer.valid_scores

,Recall@10,Unpopularity@10,Serendipity@10,Long-tail rate@10,epoch,losses
0,0.543177,42.309594,8.443820,0.280532,0,NaN
0,0.553501,42.501488,8.766321,0.284574,10,0.231352
0,0.553957,42.900253,8.779521,0.284787,20,0.229864
0,0.555565,42.223274,8.872850,0.286915,30,0.229391
0,0.558910,43.063299,9.026123,0.285426,40,0.227196
0,0.558333,42.989188,8.994074,0.289468,50,0.226781


# Result (before training)

In [ ]:
knn = searches.NearestNeighborhood(model)

In [ ]:
trainer.valid(knn, userwise)
re_pre = trainer.valid_scores.copy()

In [ ]:
if out_to_file:
    now = datetime.now()
    file_name = now.strftime("pre_%Y_%m_%d_%H%M.csv")
    re_pre.to_csv(out_dir + file_name, index=False)

display(re_pre)

# Training

In [ ]:
trainer.fit(n_batch, n_epoch)

# Result

## CML (Base)

In [ ]:
knn = searches.NearestNeighborhood(model)

In [ ]:
trainer.valid(knn, userwise)
re_base = trainer.valid_scores.copy()

In [ ]:
if out_to_file:
    now = datetime.now()
    file_name = now.strftime("base_%Y_%m_%d_%H%M.csv")
    re_base.to_csv(out_dir + file_name, index=False)

display(re_base)

# MPCMP

In [ ]:
# Hyperparameters
n_sample = 30
bias_li = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
gamma = distributions.Gamma()

In [ ]:
rows = []
for bias in bias_li:
    print(f'Prosessing... [bias = {bias}]')
    
    mp = searches.MutualProximity(model, gamma, n_sample, bias)
    
    trainer.valid(mp, userwise)
    row = trainer.valid_scores.copy()
    
    row.insert(0, 'Bias', bias)
    
    rows.append(row)

re_mpcml = pd.concat(rows)

In [ ]:
if out_to_file:
    now = datetime.now()
    file_name = now.strftime("mpcml_%Y_%m_%d_%H%M.csv")
    re_mpcml.to_csv(out_dir + file_name, index=False)

display(re_mpcml)

## CML vs MPCML

In [ ]:
# df -> list

k = 10
n_bias = len(bias_li)

# Before training
recall_pre         = re_pre[f'Recall@{k}'].tolist() * n_bias
unpopularity_pre   = re_pre[f'Unpopularity@{k}'].tolist() * n_bias
serndipity_pre     = re_pre[f'Serendipity@{k}'].tolist() * n_bias
longtale_pre       = re_pre[f'Long-tail rate@{k}'].tolist() * n_bias

# CML(Base)
recall_base        = re_base[f'Recall@{k}'].tolist() * n_bias
unpopularity_base  = re_base[f'Unpopularity@{k}'].tolist() * n_bias
serndipity_base    = re_base[f'Serendipity@{k}'].tolist() * n_bias
longtale_base      = re_base[f'Long-tail rate@{k}'].tolist() * n_bias

# MPCML
recall_mpcml       = re_mpcml[f'Recall@{k}'].tolist()
unpopularity_mpcml = re_mpcml[f'Unpopularity@{k}'].tolist()
serndipity_mpcml   = re_mpcml[f'Serendipity@{k}'].tolist()
longtale_mpcml     = re_mpcml[f'Long-tail rate@{k}'].tolist()

### Recall vs Unpopularity

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

# Recall
line_recall_base  = ax1.plot(bias_li, recall_base,  label='Recall - CML', color='tab:blue')
line_recall_mpcml = ax1.plot(bias_li, recall_mpcml, label='Recall - MPCML', marker='o', color='tab:blue')

# Unpopularity
ax2 = ax1.twinx()
line_unpop_base  = ax2.plot(bias_li, unpopularity_base,  label='Unpopularity - CML', color='tab:orange')
line_unpop_mpcml = ax2.plot(bias_li, unpopularity_mpcml, label='Unpopularity - MPCML', marker='o', color='tab:orange')

ax1.set_xticks(bias_li)
ax1.tick_params(axis='y')
ax2.tick_params(axis='y')
ax1.set_xlabel('Bias (α)')
ax1.set_ylabel('Recall')
ax2.set_ylabel('Unpopularity')

lines = [line_recall_base[0], line_recall_mpcml[0], line_unpop_base[0], line_unpop_mpcml[0]]
labels = [line.get_label() for line in lines]
fig.legend(lines, labels, bbox_to_anchor=(0.9, 0.6))

plt.grid(axis='y')

if out_to_file:
    now = datetime.now()
    file_name = now.strftime("tradeoff_%Y_%m_%d_%H%M.png")
    plt.savefig(out_dir + file_name, bbox_inches='tight')

plt.show()

### Serendipity

In [ ]:
plt.figure(figsize=(10, 6))
# plt.plot(bias_li, serndipity_pre, label='Before training)
plt.plot(bias_li, serndipity_base, label='CML')
plt.plot(bias_li, serndipity_mpcml, label='MPCML', marker='o')

plt.xticks(bias_li)
plt.xlabel('Bias (α)')
plt.ylabel('Serendipity')
plt.legend()
plt.grid(axis='y')

if out_to_file:
    now = datetime.now()
    file_name = now.strftime("serendipity_%Y_%m_%d_%H%M.png")
    plt.savefig(out_dir + file_name, bbox_inches='tight')

plt.show()

### Long-tail rate

In [ ]:
plt.figure(figsize=(10, 6))
# plt.plot(bias_li, longtale_pre, label='Before training')
plt.plot(bias_li, longtale_base, label='CML')
plt.plot(bias_li, longtale_mpcml, label='MPCML', marker='o')

plt.xticks(bias_li)
plt.xlabel('Bias (α)')
plt.ylabel('Long-tail rate')
plt.legend()
plt.grid(axis='y')

if out_to_file:
    now = datetime.now()
    file_name = now.strftime("longtail_%Y_%m_%d_%H%M.png")
    plt.savefig(out_dir + file_name, bbox_inches='tight')

plt.show()